In [2]:
import pandas as pd
import numpy as np
import requests
import io
import pandas_datareader as web
from google.colab import files

In [3]:
#Get list of companies in .csv from https://www.nasdaq.com/market-activity/stocks/screener
uploaded = files.upload()

Saving nasdaq.csv to nasdaq.csv


In [40]:
#Create the DataFrame and fill with historical data
tickers = pd.read_csv(io.BytesIO(uploaded['nasdaq.csv']))['Symbol']

df = pd.DataFrame(columns=tickers)

for symbol in tickers:
    df[symbol] = web.DataReader(symbol, "yahoo", "2013-01-01", "2020-12-29")["Adj Close"]

#Drop the rows where at least one element is missing.
df = df.dropna()
df

assets = df.columns

In [34]:
pip install PyPortfolioOpt

     |████████████████████████████████| 51kB 2.8MB/s 


In [37]:
#Optimize the portfolio
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns

In [43]:
#Calculate the expected annualized returns and the annualized ...
mu = expected_returns.mean_historical_return(df)
S = risk_models.sample_cov(df)


In [45]:
#Optimize fot the maximal Sharpe ratio
ef = EfficientFrontier(mu,S)
weights = ef.max_sharpe()

cleaned_weights = ef.clean_weights()
print(cleaned_weights)

ef.portfolio_performance(verbose=True)

OrderedDict([('AAPL', 0.0), ('ABST', 0.0), ('ACIA', 0.0), ('ACIW', 0.0), ('ACLS', 0.0), ('ADBE', 0.0), ('ADI', 0.0), ('ADP', 0.0), ('ADSK', 0.0), ('AGYS', 0.03533), ('ALGM', 0.0), ('ALLT', 0.0), ('ALRM', 0.56024), ('ALTR', 0.0), ('AMAT', 0.26014), ('AMBA', 0.06032), ('AMD', 0.0), ('AMKR', 0.0), ('AMSWA', 0.08398), ('ANSS', 0.0), ('AOSL', 0.0), ('API', 0.0), ('ASML', 0.0)])
Expected annual return: 1740.9%
Annual volatility: 33.0%
Sharpe Ratio: 52.74


(17.408876948743615, 0.32970307134446936, 52.74102202880588)

In [46]:
pip install pulp

     |████████████████████████████████| 40.6MB 107kB/s 


In [49]:
#Get the discrete allocation of each share per stock
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices

In [50]:
portfolio_val = 5000
latest_prices = get_latest_prices(df)
weights = cleaned_weights
da = DiscreteAllocation(weights, latest_prices, total_portfolio_value=portfolio_val)
allocation, leftover = da.lp_portfolio()
print('Discrete allocation: ', allocation)
print('Funds Rreaming $', leftover)

Discrete allocation:  {'AGYS': 4, 'ALRM': 28, 'AMAT': 16, 'AMBA': 3, 'AMSWA': 25}
Funds Rreaming $ 2.319974899291992


In [59]:
#Get the companies name
def get_company_name(symbol):
  url = 'http://d.yimg.com/autoc.finance.yahoo.com/autoc?query=' + symbol + '&region=1&lang=en'
  result = requests.get(url).json()
  for r in result['ResultSet']['Result']:
    if r['symbol'] == symbol:
      return r['name']

In [60]:
company_name = []
for symbol in allocation:
  company_name.append(get_company_name(symbol))

In [61]:
discrete_allocation_list = []
for symbol in allocation:
  discrete_allocation_list.append(allocation.get(symbol))

In [62]:
portfolio_df = pd.DataFrame(columns=['Company_name', 'Company_Ticker', 'Discrete_val_'+ str(portfolio_val)])

In [63]:
portfolio_df['Company_name'] = company_name
portfolio_df['Company_Ticker'] = allocation
portfolio_df['Discrete_val_'+ str(portfolio_val)] = discrete_allocation_list

In [64]:
portfolio_df

,Company_name,Company_Ticker,Discrete_val_5000
0,"Agilysys, Inc.",AGYS,4
1,"Alarm.com Holdings, Inc.",ALRM,28
2,"Applied Materials, Inc.",AMAT,16
3,"Ambarella, Inc.",AMBA,3
4,"American Software, Inc.",AMSWA,25
